#Data Generator Functions

In [0]:
%run "/Users/cabreirajm@gmail.com/DataPipelineCabreira/Helpers/general_functions"


In [0]:
#%pip install dbldatagen

from pyspark.sql.types import *
from pyspark.sql.functions import expr, col, current_timestamp
import dbldatagen as dg
import time

## 1.0 Streaming Data Function


In [0]:
@log_function_execution
def generate_api_data():
    estados = ['RO','AC','AM','RR','PA','AP','TO','MA','PI','CE','RN','PB','PE','AL','SE','BA','MG','ES','RJ','SP','PR','SC','RS','MS','MT','GO','DF']
    provedores_email = ['@gmail.com','@hotmail.com','@outlook.com','@uol.com']
    sexo_usuario = ['M','F']
    profissao_usuario = ['Engenheiro de Dados','Arquiteto de Dados', 'Analista de Dados', 'Analista de BI', 'Cientista de Dados', 'Analista de Negocio', 'Desenvolvedor de ETL', 'Desenvolvedor de Sistemas']
    cursos = [
        'f260cd97c6c9813b01601e834a2added', 
        '34bdd77f6954552d11c4f5547cb41458', 
        'c2d6bcbc3e46555bb1e7e9afbc24d3af'
    ]
    local_acesso = ['iphone', 'android','chrome','safari','firefox']
    forma_pagamento = ['boleto', 'credito','pix']
    distribuicao_forma_pagamento = [25, 70, 5]
    disconto = [0.05,0.1,0.15]
    distribuicao_disconto = [50,35,15]
    parcelas = [1,2,3,10,12]
    distribuicao_parcelas = [20,15,5,30,30]
    idade = [18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
    number_unique_rows = 1000

    df_test = (
        dg.DataGenerator(spark, 
                        name="data_set", 
                        rows=number_unique_rows,
                        uniqueValues=number_unique_rows,
                        partitions=8, 
                        randomSeedMethod='hash_fieldname')
        .withIdOutput()

        .withColumn("id_venda", 
                    LongType(), 
                    minValue=0,
                    uniqueValues=number_unique_rows, 
                    omit=True)
        
        .withColumn("access_date", 
                    "timestamp",
                    # omit=True,
                    begin="2024-06-01 00:00:00",
                    end="2025-12-31 23:59:59",
                    interval="247 seconds", 
                    baseColumn=["id_venda"])
        
        .withColumn("ip_address", 
                    StringType(),
                    # omit=True, 
                    template=r"\n.\n.\n.\n",
                    baseColumn=["id_venda"])
        
        .withColumn("access_point", 
                    StringType(),
                    # omit=True,
                    values=local_acesso,
                    baseColumn=["id_venda"])
        
        .withColumn("hash", 
                    StringType(),
                    omit=True, 
                    expr="md5(concat(cast(rand() as string),id_venda,access_date,ip_address,access_point))",
                    baseColumn=["id_venda","access_date","ip_address","access_point"])
        
        .withColumn("nome_usuario", 
                    StringType(),
                    omit=True, 
                    expr="concat('Usuario ', concat(left(hash,4),substr(hash,14,2),right(hash,4)))",
                    baseColumn=["hash"])
        
        .withColumn("provedor_email", 
                    "string", 
                    random=True, 
                    omit=True, 
                    values=provedores_email)
        
        .withColumn("email_usuario", 
                    StringType(),
                    omit=True, 
                    expr="concat('usuario_', concat(left(hash,4),substr(hash,14,2),right(hash,4)), provedor_email)",
                    baseColumn=["hash"])
        
        .withColumn("sexo", 
                    "string", 
                    random=True, 
                    omit=True, 
                    values=sexo_usuario)
        
        .withColumn("profissao", 
                    "string", 
                    random=True, 
                    omit=True, 
                    values=profissao_usuario)
        
        .withColumn("idade", 
                    "string", 
                    random=True, 
                    omit=True, 
                    values=idade)
        
        .withColumn("estado", 
                    StringType(),
                    omit=True, 
                    values=estados,
                    baseColumn="id_venda")
        
        .withColumn("product_uuid", 
                    StringType(),
                    omit=True, 
                    values=cursos,
                    baseColumn="id_venda")
        
        .withColumn("valor", 
                    StringType(),
                    omit=True, 
                    expr="case when product_uuid = 'f260cd97c6c9813b01601e834a2added' then 589.90 when product_uuid = '34bdd77f6954552d11c4f5547cb41458' then 659.90 when product_uuid = 'c2d6bcbc3e46555bb1e7e9afbc24d3af' then 549.90 end",
                    baseColumn=["id_venda"])
        
        .withColumn("valor_string", 
                    StringType(),
                    omit=True, 
                    expr="case when product_uuid = 'f260cd97c6c9813b01601e834a2added' then 'R$ 589,90' when product_uuid = '34bdd77f6954552d11c4f5547cb41458' then 'R$ 659,90' when product_uuid = 'c2d6bcbc3e46555bb1e7e9afbc24d3af' then 'R$ 549,90' end",
                    baseColumn=["id_venda"])
        
        .withColumn("disconto", 
                    StringType(),
                    omit=True, 
                    values=disconto, 
                    nullable=True, 
                    percentNulls=0.7, 
                    weights=distribuicao_disconto,
                    baseColumn="id_venda")
        
        .withColumn("valor_disconto", 
                    StringType(),
                    omit=True, 
                    expr="cast((valor * disconto) as numeric(9,2))",
                    baseColumn=["id_venda",'valor', 'disconto'])
        
        .withColumn("valor_pago", 
                    StringType(),
                    omit=True, 
                    expr="cast((valor - coalesce(valor_disconto,0)) as numeric(9,2))",
                    baseColumn=["id_venda",'valor_disconto', 'valor'])
        
        .withColumn("forma_pagamento", 
                    StringType(),
                    omit=True, 
                    values=forma_pagamento, 
                    weights=distribuicao_forma_pagamento,
                    baseColumn="id_venda")
        
        .withColumn("quantidade_parcelas_base", 
                    StringType(),
                    omit=True, 
                    values=parcelas, 
                    weights=distribuicao_parcelas,
                    baseColumn="id_venda")
        
        .withColumn("quantidade_parcelas", 
                    StringType(),
                    omit=True, 
                    expr="case when forma_pagamento in ('boleto','pix') then 1 else quantidade_parcelas_base end",
                    baseColumn=["id_venda",'quantidade_parcelas_base','forma_pagamento'])
        
        .withColumn("valor_parcelas", 
                    StringType(),
                    omit=True, 
                    expr="cast((valor_pago/quantidade_parcelas) as numeric(9,2))",
                    baseColumn=["id_venda",'quantidade_parcelas','valor_pago'])
        
        .withColumn("info_usuario",
                    StructType([
                        StructField('nome',StringType()),
                        StructField('idade',StringType()),
                        StructField('sexo',StringType()),
                        StructField('email',StringType()),
                        StructField('profissao',StringType()),
                        StructField('estado',StringType())
                    ]),
                    omit=True, 
                    expr="""
                        named_struct(
                            'nome', nome_usuario, 
                            'idade', idade,
                            'sexo', sexo,
                            'email', email_usuario,
                            'profissao', profissao,
                            'estado', estado
                        )""",
                    nullable=True, 
                    percentNulls=0.7,
                    baseColumn=['nome_usuario','idade','sexo','email_usuario','profissao', 'estado'])
        
        .withColumn("info_produto",
                    StructType([
                        StructField('product_uuid',StringType()),
                        StructField('valor',StringType())
                    ]),
                    omit=True, 
                    expr="""
                        case when info_usuario is not null then 
                        named_struct(
                            'product_uuid', product_uuid, 
                            'valor', valor_string
                        )
                        else null end""",
                    nullable=True, 
                    percentNulls=0.7,
                    baseColumn=['info_usuario','product_uuid','valor_string'])
        
        .withColumn("info_pagamento",
                    StructType([
                        StructField('disconto',StringType()),
                        StructField('valor',StringType()),
                        StructField('forma_pagamento',StringType()),
                        StructField('quantidade_parcelas',StringType()),
                        StructField('valor_parcelas',StringType())
                    ]),
                    omit=True, 
                    expr="""
                        case when info_produto is not null then 
                        named_struct(
                            'disconto', disconto, 
                            'valor', valor_pago,
                            'forma_pagamento', forma_pagamento,
                            'quantidade_parcelas', quantidade_parcelas,
                            'valor_parcela', valor_parcelas
                        ) else null end""",
                    baseColumn=['info_produto','disconto','valor_pago','forma_pagamento','quantidade_parcelas','valor_parcelas'])
        
        .withColumn("payload",
                    StructType([
                        StructField('info_usuario',StructType([
                                                        StructField('nome',StringType()),
                                                        StructField('idade',StringType()),
                                                        StructField('sexo',StringType()),
                                                        StructField('email',StringType()),
                                                        StructField('profissao',StringType()),
                                                        StructField('estado',StringType())
                                                    ])),
                        StructField('info_produto',StructType([
                                                        StructField('product_uuid',StringType()),
                                                        StructField('valor',StringType())
                                                    ])),
                        StructField('info_pagamento',StructType([
                                                        StructField('disconto',StringType()),
                                                        StructField('valor',StringType()),
                                                        StructField('forma_pagamento',StringType()),
                                                        StructField('quantidade_parcelas',StringType()),
                                                        StructField('valor_parcelas',StringType())
                                                    ]))
                    ]),
                    expr="""
                        named_struct(
                            'info_usuario', info_usuario,
                            'info_produto', info_produto,
                            'info_pagamento', info_pagamento
                        )""",
                    baseColumn=['info_usuario','info_produto','info_pagamento'])
    )

    df = df_test.build(withStreaming=True, options={'rowsPerSecond': 387}).drop('id')
    
    generate_api_stream_data = (
        df.writeStream
            .format('json')
            .outputMode('append')
            .queryName('generate_api_stream_data')
            .option('checkpointLocation','dbfs:/FileStore/landing/_checkpoint')
            .start("dbfs:/FileStore/landing/stream")
    )
    
    return generate_api_stream_data


## 2.0 Batch Data Function

In [0]:
@log_function_execution
def generate_files_data(number_rows):
    estados = ['RO','AC','AM','RR','PA','AP','TO','MA','PI','CE','RN','PB','PE','AL','SE','BA','MG','ES','RJ','SP','PR','SC','RS','MS','MT','GO','DF']
    sexo_usuario = ['M','F']
    profissao_usuario = ['Engenheiro de Dados','Arquiteto de Dados', 'Analista de Dados', 'Analista de BI', 'Cientista de Dados', 'Analista de Negocio', 'Desenvolvedor de ETL', 'Desenvolvedor de Sistemas']
    cursos = [
        'Construindo o seu Primeiro Pipeline de Dados com o Databricks', 
        'Do Primeiro Pipeline ao Data Lakehouse com o Databricks', 
        'Construindo Pipelines de Dados usando o Spark Structured Streaming'
    ]
    empresas = ['Empresa A', 'Empresa B', 'Empresa C', 'Empresa D','Empresa E','Empresa F','Empresa G','Empresa H','Empresa I','Empresa J','Empresa L','Empresa M','Empresa N']
    distribuicao_empresas = [30,5,3,2,10,1,5,15,1,8,5,12,7]
    disconto = ['5%','10%','15%']
    idade = [18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
    number_unique_rows = number_rows

    df_test = (
        dg.DataGenerator(spark, 
                        name="data_set", 
                        rows=number_unique_rows,
                        uniqueValues=number_unique_rows,
                        partitions=8, 
                        randomSeedMethod='hash_fieldname')
        .withIdOutput()

        .withColumn("id_venda", 
                    LongType(), 
                    minValue=0,
                    uniqueValues=number_unique_rows, 
                    omit=True)
        
        .withColumn("data_venda", 
                    "date",
                    begin="2024-06-01",
                    end="2025-12-31",
                    interval="10 days", 
                    random=True,
                    baseColumn=["id_venda"])
        
        .withColumn("nome_empresa", 
                    StringType(),
                    values=empresas,
                    weights=distribuicao_empresas,
                    baseColumn=["id_venda"])
        
        .withColumn("sexo", 
                    "string", 
                    random=True, 
                    values=sexo_usuario)
        
        .withColumn("hash", 
                    StringType(),
                    omit=True, 
                    expr="md5(concat(cast(rand() as string),id_venda,data_venda,nome_empresa,sexo))",
                    baseColumn=["id_venda","data_venda","nome_empresa","sexo"])
        
        .withColumn("nome_funcionario", 
                    StringType(), 
                    expr="concat('Funcionario ',concat(left(hash,4),substr(hash,14,2),right(hash,4)))",
                    baseColumn=["hash"])
        
        .withColumn("provedor_email", 
                    "string", 
                    omit=True, 
                    expr="concat('@',replace(lower(nome_empresa),' ',''),'.com.br')",
                    baseColumn=["nome_empresa"])
        
        .withColumn("email_functionario", 
                    StringType(),
                    expr="concat(lower(replace(nome_funcionario,' ','_')), provedor_email)",
                    baseColumn=["nome_funcionario","provedor_email"])
        
        .withColumn("profissao", 
                    "string", 
                    random=True, 
                    values=profissao_usuario)
        
        .withColumn("idade", 
                    "string", 
                    random=True, 
                    values=idade)
        
        .withColumn("estado", 
                    StringType(),
                    values=estados,
                    baseColumn="id_venda")
        
        .withColumn("curso", 
                    StringType(),
                    values=cursos,
                    baseColumn="id_venda")
        
        .withColumn("valor", 
                    StringType(),
                    expr="case when curso = 'Construindo o seu Primeiro Pipeline de Dados com o Databricks' then 'R$ 789,90' when curso = 'Do Primeiro Pipeline ao Data Lakehouse com o Databricks' then 'R$ 689,90' when curso = 'Construindo Pipelines de Dados usando o Spark Structured Streaming' then 'R$ 549,90' end",
                    baseColumn=["id_venda"])
        
        .withColumn("disconto", 
                    StringType(),
                    expr="case when nome_empresa in ('Empresa A', 'Empresa G','Empresa H') then '5%' when nome_empresa in ('Empresa F','Empresa D','Empresa E') then '10%' else '15%' end",
                    baseColumn=["id_venda","nome_empresa"])
    )

    df = df_test.build().drop('id')

    df.coalesce(1).write.format('csv').mode('append').option('header',True).option('sep',',').save('dbfs:/FileStore/landing/files')

    exclusion_list = ['_committed','_started','_SUCCESS']
    for file_to_be_excluded in exclusion_list:
        for file in dbutils.fs.ls('dbfs:/FileStore/landing/files'):
            if file_to_be_excluded in file.name:
                dbutils.fs.rm(f'dbfs:/FileStore/landing/files/{file.name}')

    spark.read.text(f'dbfs:/FileStore/landing/files/{file.name}').limit(2)

    return print(f"O arquivo .csv com {number_rows} registros foi gerado no diretorio 'dbfs:/FileStore/landing/files'.")